# Phylogenetic Compositional Tensor Factorization (phylo-CTF) Introduction

This tutorial builds upon the CTF tutorial and explains how you can also perform CTF weighted by phylogenetic information. If you would like to better understand CTF please first complete that tutorial. In this tutorial we will use Phylogenetic Compositional Tensor Factorization (phylo-CTF) which we provide in the software package [gemelli](https://github.com/biocore/gemelli). Phylo-CTF can account for repeated measures, compositionality, and sparsity in microbiome data.

In this tutorial we use _gemelli_ to perform phylo-CTF on a time series dataset comparing Crohn's and control subjects over a period of 25 weeks published in [Vázquez-Baeza et al](https://gut.bmj.com/content/67/9/1743). First we will download the processed data originally from [here](https://qiita.ucsd.edu/study/description/2538#). This data can be downloaded with the following links:

* **Table** (table.qza) | [download](https://github.com/biocore/gemelli/tree/master/ipynb/tutorials/IBD-2538/data/table.qza)
* **Rarefied Table** (rarefied-table.qza) | [download](https://github.com/biocore/gemelli/tree/master/ipynb/tutorials/IBD-2538/data/rarefied-table.qza)
* **Sample Metadata** (metadata.tsv) | [download](https://github.com/biocore/gemelli/tree/master/ipynb/tutorials/IBD-2538/data/metadata.tsv)
* **Feature Metadata** (taxonomy.qza) | [download](https://github.com/biocore/gemelli/tree/master/ipynb/tutorials/IBD-2538/data/taxonomy.qza)
* **Tree** (sepp-insertion-tree.qza) | [download](https://github.com/biocore/gemelli/tree/master/ipynb/tutorials/IBD-2538/data/sepp-insertion-tree.qza)

**Note**: This tutorial assumes you have installed [QIIME2](https://qiime2.org/) using one of the procedures in the [install documents](https://docs.qiime2.org/2020.2/install/). This tutorial also assumed you have installed, [Qurro](https://github.com/biocore/qurro), [DEICODE](https://github.com/biocore/DEICODE), and [gemelli](https://github.com/biocore/gemelli).

First, we will make a tutorial directory and download the data above and move the files to the `IBD-2538/data` directory:

```bash
mkdir IBD-2538
```
```bash
# move downloaded data here
mkdir IBD-2538/data
```

Next we will import our data with the QIIME2 Python API. 


In [1]:
!mkdir IBD-2538
!mkdir IBD-2538/core-metric-output

mkdir: IBD-2538/core-metric-output: File exists


In order to account for the correlation among samples from the same subject we will employ phylogenetic compositional tensor factorization (CTF). CTF builds on the ability to account for compositionality and sparsity using the robust center log-ratio transform covered in the RPCA tutorial (found [here](https://forum.qiime2.org/t/robust-aitchison-pca-beta-diversity-with-deicode)) but restructures and factors the data as a tensor. Here we will run CTF through _gemelli_ and explore/interpret the different results. Additionally, phylogenetic CTF incorporates internal nodes and branch lengths of a tree into the factorization. 

There are two forms of phylogenetic CTF bieng (1) with taxonomy information and (2) without. Here we will run phylogenetic-ctf-with-taxonomy but it can also be run without taxonomy called just phylogenetic-ctf.

To run phylo-CTF we only need to run one command (gemelli phylogenetic-ctf-with-taxonomy). The required input requirements are:

1. table
    - The table is of type `FeatureTable[Frequency]` which is a table where the rows are features (e.g. ASVs/microbes), the columns are samples, and the entries are the number of sequences for each sample-feature pair.
2. tree
    - This is a phylogeny of type `Phylogeny[Rooted]` where all the features in the `table` are represented in the tree. 
3. feature-metadata-file
    - This is a metadata file (e.g. tsv, or `FeatureTable[Taxonomy]` .qza) where the rows are matched to the table features and the columns are feature metadata such as taxonomy, gene pathway, etc... 
4. sample-metadata
    - This is a QIIME2 formatted [metadata](https://docs.qiime2.org/2020.2/tutorials/metadata/) (e.g. tsv format) where the rows are samples matched to the (1) table and the columns are different sample data (e.g. time point). 
5. individual-id-column
    - This is the name of the column in the (2) metadata that indicates the individual subject/site (e.g. subject ID) that was sampled repeatedly.
6. state-column
    - This is the name of the column in the (2) metadata that indicates the numeric repeated measure (e.g., Time in months/days) or non-numeric category (i.e. decade/body-site). 
7. output-dir
    - The desired location of the output. We will cover each output independently below.  


In this tutorial our individual-id-column is `host_subject_id` and our state-column is different time points denoted as `timepoint` in the sample metadata. Now we are ready to run phylo-CTF:


In [19]:
!qiime fragment-insertion filter-features\
    --i-table IBD-2538/data/table.qza  \
    --i-tree IBD-2538/data/sepp-insertion-tree.qza  \
    --output-dir IBD-2538/data/matched-table

!qiime gemelli phylogenetic-ctf-with-taxonomy\
    --i-table  IBD-2538/data/matched-table/filtered_table.qza\
    --i-phylogeny  IBD-2538/data/sepp-insertion-tree.qza\
    --m-sample-metadata-file IBD-2538/data/metadata.tsv\
    --m-taxonomy-file IBD-2538/data/taxonomy.qza\
    --p-state-column timepoint\
    --p-individual-id-column host_subject_id\
    --p-min-feature-frequency 10\
    --output-dir IBD-2538/ctf-results


Saved FeatureTable[Frequency] to: IBD-2538/data/matched-table/filtered_table.qza
Saved FeatureTable[Frequency] to: IBD-2538/data/matched-table/removed_table.qza
Saved PCoAResults % Properties('biplot') to: IBD-2538/ctf-results/subject_biplot.qza
Saved PCoAResults % Properties('biplot') to: IBD-2538/ctf-results/state_biplot.qza
Saved DistanceMatrix to: IBD-2538/ctf-results/distance_matrix.qza
Saved SampleData[SampleTrajectory] to: IBD-2538/ctf-results/state_subject_ordination.qza
Saved FeatureData[FeatureTrajectory] to: IBD-2538/ctf-results/state_feature_ordination.qza
Saved Phylogeny[Rooted] to: IBD-2538/ctf-results/counts_by_node_tree.qza
Saved FeatureTable[Frequency] to: IBD-2538/ctf-results/counts_by_node.qza
Saved FeatureData[Taxonomy] to: IBD-2538/ctf-results/t2t_taxonomy.qza
Saved FeatureTable[Frequency] to: IBD-2538/ctf-results/subject_table.qza



We will now cover the output files:
* subject_biplot
* state_biplot
* distance_matrix
* state_subject_ordination
* state_feature_ordination
* counts_by_node_tree
* counts_by_node
* t2t_taxonomy
* subject_table


First, we will explore the `subject_biplot` which is an ordination where dots represent _subjects_ not _samples_ and arrows represent features (e.g. ASVs). First, we will need to aggregate the metadata by subject (i.e. collapsing the metadata of all samples from a given subject). This can be done by hand or using DataFrames in python (with pandas) or R like so:


In [20]:
import pandas as pd
from qiime2 import Metadata
import numpy as np
from biom import Table
from skbio import OrdinationResults

# first we import the metdata into pandas
mf = pd.read_csv('IBD-2538/data/metadata.tsv', sep='\t',index_col=0)
# next we aggregate by subjects (i.e. 'host_subject_id') 
# and keep the first instance of 'diagnosis_full' by subject.
mf = mf.groupby('host_subject_id').agg({'ibd':'first','active_disease':'first'})
# now we save the metadata in QIIME2 format.
mf.index.name = '#SampleID'
mf.to_csv('IBD-2538/data/subject-metadata.tsv', sep='\t')
mf.head(5)


,ibd,active_disease
#SampleID,,
s1000100,Control,quiescent
s1000200,Control,quiescent
s1000300,Control,quiescent
s1000500,Control,quiescent
s1000600,Control,quiescent


Unlike conventional CTF, the arrows in the phylo-CTF biplot can be both internal nodes and features of the table. In order to label the taxonomy we use the lowest common ancestor of that internal node. To do this we feed the `t2t_taxonomy` output from phylo-CTF into the biplot. Additionally, we can use Empress to generate plots combining both the tree and biplot view at one time. This view can help us understand what phylogenetic partitions separate our samples along a PC axis. 

Phylo-CTF output contains a table that allows us to use the internal nodes of the tree as possible log-ratios (i.e. `counts_by_node`). In this table the internal nodes contain the sum of all the counts up to that node. `Note,` it is important not to take the log-ratio of two internal nodes in the tree so we will visualize the loadings on the tree using [Empress](https://github.com/biocore/empress).

The log-ratio can then be taken with [Qurro](https://github.com/biocore/qurro) to explore the feature loading partitions (arrows) in this biplot as a log-ratio of the original table counts. This allows us to relate these low-dimensional representations back to our original data. Additionally, log-ratios provide a nice set of data points for additional analysis such as LME models. 


In [22]:
!qiime empress community-plot\
    --i-tree IBD-2538/ctf-results/counts_by_node_tree.qza\
    --i-feature-table IBD-2538/ctf-results/subject_table.qza\
    --i-pcoa IBD-2538/ctf-results/subject_biplot.qza\
    --m-sample-metadata-file IBD-2538/data/subject-metadata.tsv\
    --m-feature-metadata-file IBD-2538/ctf-results/t2t_taxonomy.qza\
    --p-filter-missing-features\
    --p-number-of-features 50\
    --o-visualization IBD-2538/ctf-results/empress.qzv

!qiime qurro loading-plot\
    --i-ranks IBD-2538/ctf-results/subject_biplot.qza\
    --i-table IBD-2538/ctf-results/counts_by_node.qza\
    --m-sample-metadata-file IBD-2538/data/metadata.tsv\
    --m-feature-metadata-file  IBD-2538/ctf-results/t2t_taxonomy.qza\
    --o-visualization IBD-2538/ctf-results/qurro.qza



Saved Visualization to: IBD-2538/ctf-results/empress.qzv
Saved Visualization to: IBD-2538/ctf-results/qurro.qza.qzv


From this visualization we can see that the Crohn's subjects clearly separate from the healthy controls in the ordination on the right. We can also observe, by adding a barplot of the PC2 loadings (see the [empress tutorial](https://github.com/biocore/empress) for more info on how to make these plots). 

![image.png](etc/ctf_empress_plot_one.png)

One additional benefit to having the phylogeny is the ability to explore ratios between phylogenetic paritions. For example the nodes n58 and n52 within the Bacteroidetes phylum which represent Bacteroides and Prevotella genera respectively. 


![image.png](etc/ctf_empress_plot_two.png)

![image.png](etc/ctf_empress_plot_three.png)

We can then obtain the log-ratio based between these nodes from the qurro visualization.


![image.png](etc/ctf_phylo_qurro.png)


We can then export the log-ratio output and plot the data.


In [23]:
import pandas as pd

# import log-ratio data
metadata_one = pd.read_csv('IBD-2538/data/metadata.tsv',
                           sep='\t', index_col=0)
# import rest of the metadata
metadata_two = pd.read_csv('IBD-2538/ctf-results/sample_plot_data.tsv',
                           sep='\t', index_col=0)[['Current_Natural_Log_Ratio']]
# merge the data
log_ratio_metdata = pd.concat([metadata_two, metadata_one], axis=1)
# ensure no duplicate columns
log_ratio_metdata = log_ratio_metdata.dropna(subset=['Current_Natural_Log_Ratio'])
log_ratio_metdata.index = log_ratio_metdata.index.astype(str)
# export in QIIME2 format
log_ratio_metdata = log_ratio_metdata[['timepoint','host_subject_id',
                                       'ibd','Current_Natural_Log_Ratio']]

log_ratio_metdata.index.name = '#SampleID'
log_ratio_metdata.to_csv('IBD-2538/ctf-results/merged_sample_plot_data.tsv', sep='\t')
log_ratio_metdata.head(2)


,timepoint,host_subject_id,ibd,Current_Natural_Log_Ratio
#SampleID,,,,
2538.1000102,25,s1000100,Control,4.663439
2538.1000104000004,6,s1000100,Control,0.470004



As you can see above the metadata now has the added column of `Current_Natural_Log_Ratio` from Qurro. So now we will continue to explore this log-ratio by first plotting it explicitly over time with q2-longitudinal.


In [24]:
!qiime longitudinal volatility \
    --m-metadata-file IBD-2538/ctf-results/merged_sample_plot_data.tsv\
    --p-state-column timepoint \
    --p-individual-id-column host_subject_id \
    --p-default-group-column ibd \
    --p-default-metric Current_Natural_Log_Ratio \
    --o-visualization IBD-2538/ctf-results/log_ratio_plot.qzv


Saved Visualization to: IBD-2538/ctf-results/log_ratio_plot.qzv



This demonstrates that we can recreate the separation by IBD that we saw in both the `subject_biplot` & `state_subject_ordination`, allowing us to associate specific taxa (in the numerator or denominator) with a particular phenotype.

![image.png](etc/ctf_phylo-lr-vol.png)

We can test the statistical power of this log-ratio to differentiate samples by IBD status using a linear mixed effects (LME) through q2-longitudinal. 


In [ ]:
!qiime longitudinal linear-mixed-effects\
    --m-metadata-file IBD-2538/ctf-results/merged_sample_plot_data.tsv\
    --p-state-column timepoint \
    --p-individual-id-column host_subject_id \
    --p-group-columns ibd \
    --p-metric Current_Natural_Log_Ratio \
    --o-visualization IBD-2538/ctf-results/lme_log_ratio.qzv


From this LME model we can see that indeed the IBD grouping is significant across time. 

![image.png](etc/ctf_lme-logratio-phylo.png)



